Copyright (c) 2017-2018 Andrew Johnson, Dan Kotlyar, Stefano Terlizzi, Gavin Ridley, GTRC

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# `Branching Reader`
## Basic Operation

This notebook demonstrates the capability of the [`serpentTools`](https://github.com/CORE-GATECH-GROUP/serpent-tools) package to read branching coefficient files. The format of these files is structured to iterate over:

1. Branch states, e.g. burnup, material properties
1. Homogenized universes
1. Group constant data

The output files are described in more detail on the [SERPENT Wiki](http://serpent.vtt.fi/mediawiki/index.php/Automated_burnup_sequence#Output_format)

The simplest way to read these files is using the `serpentTools.read` method

In [1]:
import serpentTools
branchFile = 'demo.coe'

**Note**

Without modifying the settings, the `BranchingReader` assumes that all group constant data is presented without the associated uncertainties. See below for examples on the various ways to adjust the UserSettings

In [2]:
r0 = serpentTools.read(branchFile)

INFO    : serpentTools: Inferred reader for demo.coe: BranchingReader
INFO    : serpentTools: Preparing to read demo.coe
INFO    : serpentTools: Done reading branching file


The branches are stored in custom `BranchContainer` objects in the `branches` dictionary

In [3]:
r0.branches

{('B1000',
  'FT1200'): <serpentTools.objects.containers.BranchContainer at 0x7f17971f6ef0>,
 ('B1000',
  'FT600'): <serpentTools.objects.containers.BranchContainer at 0x7f1796951400>,
 ('B1000',
  'nom'): <serpentTools.objects.containers.BranchContainer at 0x7f17971de9e8>,
 ('B750',
  'FT1200'): <serpentTools.objects.containers.BranchContainer at 0x7f17971ee7f0>,
 ('B750',
  'FT600'): <serpentTools.objects.containers.BranchContainer at 0x7f1796947cc0>,
 ('B750',
  'nom'): <serpentTools.objects.containers.BranchContainer at 0x7f17971d7358>,
 ('nom',
  'FT1200'): <serpentTools.objects.containers.BranchContainer at 0x7f17971e80f0>,
 ('nom',
  'FT600'): <serpentTools.objects.containers.BranchContainer at 0x7f17969405c0>,
 ('nom',
  'nom'): <serpentTools.objects.containers.BranchContainer at 0x7f17971bfcc0>}

Here, the keys are tuples of strings indicating what perturbations/branch states were applied for each `SERPENT` solution. Examining a particular case

In [4]:
b0 = r0.branches['B1000', 'FT600']
print(b0)

<BranchContainer for B1000, FT600 from demo.coe>


`SERPENT` allows the user to define variables for each branch through:

`
var V1_name V1_value
`

cards. These are stored in the `stateData` attribute

In [5]:
b0.stateData

{'BOR': '1000',
 'DATE': '17/12/19',
 'TFU': '600',
 'TIME': '09:48:54',
 'VERSION': '2.1.29'}

The keys `'DATE'`, `'TIME'`, and `'VERSION'` are included by default in the output, while the `'BOR'` and `'TFU'` have been defined for this branch.

### Group Constant Data
**Note**: Group constants are converted from `SERPENT_STYLE` to `mixedCase` to fit the overall style of the project.

The `BranchContainer` stores group constant data in `HomogUniv` objects in the `universes` dictionary. 

In [6]:
for key in b0.universes:
    print(key)

(40, 1.0, 2)
(0, 10.0, 3)
(40, 0.0, 1)
(0, 1.0, 2)
(0, 0.0, 1)
(10, 10.0, 3)
(20, 1.0, 2)
(30, 1.0, 2)
(10, 1.0, 2)
(20, 0.0, 1)
(30, 0.0, 1)
(10, 0.0, 1)
(30, 10.0, 3)
(40, 10.0, 3)
(20, 10.0, 3)


The keys here are vectors indicating the universe ID, burnup, and burnup index corresponding to the point in the burnup schedule. `SERPENT` prints negative values of burnup to indicate units of days, which is reflected in the `hasDays` attribute. `hasDays-> True` indicates that the values of burnup, second item in the above tuple, are in terms of days, not MWd/kgU.

These universes can be obtained by indexing this dictionary, or by using the `getUniv` method.

In [7]:
univ0 = b0.universes[0, 1, 2]
print(univ0)
print(univ0.name)
print(univ0.bu)
print(univ0.step)
print(univ0.day)
print(b0.hasDays)

<HomogUniv 0>
0
1.0
2
None
False


Since the coefficient files do not store the day value of burnup, all `HomogUniv` objects created by the `BranchContainers` default to day zero.

In [8]:
univ1 = b0.getUniv(0, burnup=1)
univ2 = b0.getUniv(0, index=2)
assert univ0 is univ1 is univ2

Group constant data is stored in five dictionaries:

1. `infExp`: Expected values for infinite medium group constants
1. `infUnc`: Relative uncertainties for infinite medium group constants
1. `b1Exp`: Expected values for leakge-corrected group constants
1. `b1Unc`: Relative uncertainties for leakge-corrected group constants
1. `metaData`: items that do not fit the in the above categories, such as energy group structure, k-infinity, etc.

For this problem, the coefficient file does not have uncertainties, nor these metadata arguments. For this reason, the `infUnc`, `b1Unc`, and `metaData` dictionaries are emtpy.

In [9]:
univ0.infExp

{'infDiffcoef': array([ 1.83961 ,  0.682022]),
 'infFiss': array([ 0.00271604,  0.059773  ]),
 'infRem': array([], dtype=float64),
 'infS0': array([ 0.298689  ,  0.00197521,  0.00284247,  0.470054  ]),
 'infS1': array([ 0.0847372 ,  0.00047366,  0.00062865,  0.106232  ]),
 'infTot': array([ 0.310842,  0.618286])}

In [10]:
univ0.infUnc

{}

In [11]:
univ0.b1Exp

{'b1Diffcoef': array([ 1.79892 ,  0.765665]),
 'b1Fiss': array([ 0.00278366,  0.0597712 ]),
 'b1Rem': array([], dtype=float64),
 'b1S0': array([ 0.301766  ,  0.0021261 ,  0.00283866,  0.470114  ]),
 'b1S1': array([ 0.0856397 ,  0.00051071,  0.00062781,  0.106232  ]),
 'b1Tot': array([ 0.314521,  0.618361])}

In [12]:
univ0.metadata

{}

Group constants and their associated uncertainties can be obtained using the `HomogUniv.get` method.

In [13]:
univ0.get('infFiss')

array([ 0.00271604,  0.059773  ])

In [14]:
try:
    univ0.get('infS0', uncertainty=True)
except KeyError as ke:  # no uncertainties here
    print(str(ke))

'Variable infS0 absent from uncertainty dictionary'


## Iteration
The branching reader has a `iterBranches` method that works to yield branch names and their associated `BranchContainer` objects. This can be used to efficiently iterate over all the branches presented in the file.

In [15]:
for names, branch in r0.iterBranches():
    print(names, branch)

('B750', 'FT600') <BranchContainer for B750, FT600 from demo.coe>
('B1000', 'nom') <BranchContainer for B1000, nom from demo.coe>
('nom', 'FT600') <BranchContainer for nom, FT600 from demo.coe>
('B1000', 'FT600') <BranchContainer for B1000, FT600 from demo.coe>
('B1000', 'FT1200') <BranchContainer for B1000, FT1200 from demo.coe>
('B750', 'FT1200') <BranchContainer for B750, FT1200 from demo.coe>
('nom', 'FT1200') <BranchContainer for nom, FT1200 from demo.coe>
('nom', 'nom') <BranchContainer for nom, nom from demo.coe>
('B750', 'nom') <BranchContainer for B750, nom from demo.coe>


## User Control
The `SERPENT` [`set coefpara`](http://serpent.vtt.fi/mediawiki/index.php/Input_syntax_manual#set_coefpara) card already restricts the data present in the coeffient file to user control, and the `BranchingReader` includes similar control.
Below are the various settings that the `BranchingReader` uses to read and process coefficient files.

In [16]:
import six
from serpentTools.settings import rc

In [17]:
from serpentTools.settings import rc, defaultSettings
for setting in defaultSettings:
    if 'xs' in setting or 'branching' in setting:
        print(setting)
        for k, v in six.iteritems(defaultSettings[setting]):
            print('\t', k+':', v)

branching.floatVariables
	 description: Names of state data variables to convert to floats for each branch
	 type: <class 'list'>
	 default: []
xs.variableGroups
	 description: Name of variable groups from variables.yaml to be expanded into SERPENT variable to be stored
	 type: <class 'list'>
	 default: []
branching.intVariables
	 description: Name of state data variables to convert to integers for each branch
	 type: <class 'list'>
	 default: []
xs.getInfXS
	 description: If true, store the infinite medium cross sections.
	 type: <class 'bool'>
	 default: True
branching.areUncsPresent
	 description: True if the values in the .coe file contain uncertainties
	 type: <class 'bool'>
	 default: False
xs.variableExtras
	 description: Full SERPENT name of variables to be read
	 type: <class 'list'>
	 default: []
xs.getB1XS
	 description: If true, store the critical leakage cross sections.
	 type: <class 'bool'>
	 default: True


In our example above, the `BOR` and `TFU` variables represented boron concentration and fuel temperature, and can easily be cast into numeric values using the `branching.intVariables` and `brancing.floatVariables` settings. From the previous example, we see that the default action is to store all state data variables as strings.

In [18]:
assert isinstance(b0.stateData['BOR'], str)

As demonstrated in the [Settings example](https://github.com/CORE-GATECH-GROUP/serpent-tools/blob/master/examples/Settings.ipynb), use of `xs.variableGroups` and `xs.variableExtras` controls what data is stored on the `HomogUniv` objects. By default, all variables present in the coefficient file are stored.

In [19]:
rc['branching.floatVariables'] = ['BOR']
rc['branching.intVariables'] = ['TFU']
rc['xs.getB1XS'] = False
rc['xs.variableExtras'] = ['INF_TOT', 'INF_SCATT0']
r1 = serpentTools.read(branchFile)

INFO    : serpentTools: Inferred reader for demo.coe: BranchingReader
INFO    : serpentTools: Preparing to read demo.coe
INFO    : serpentTools: Done reading branching file


In [20]:
b1 = r1.branches['B1000', 'FT600']

In [21]:
b1.stateData

{'BOR': 1000.0,
 'DATE': '17/12/19',
 'TFU': 600,
 'TIME': '09:48:54',
 'VERSION': '2.1.29'}

In [22]:
assert isinstance(b1.stateData['BOR'], float)
assert isinstance(b1.stateData['TFU'], int)

Inspecting the data stored on the homogenized universes reveals only the variables explicitly requested are present 

In [23]:
univ4 = b1.getUniv(0, 0)
univ4.infExp

{'infTot': array([ 0.313338,  0.54515 ])}

In [24]:
univ4.b1Exp

{}

## Conclusion
The `BranchingReader` is capable of reading coefficient files created by the `SERPENT` automated branching process. The data is stored according to the branch parameters, universe information, and burnup. This reader also supports user control of the processing by selecting what state parameters should be converted from strings to numeric types, and further down-selection of data.